In [137]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns


## CLEANING DATASET

In [138]:
df = pd.read_csv('scraped_data_alonhadat_onlyfront.csv',encoding='utf-8')

In [139]:
df = df.drop_duplicates()

In [141]:
df

,price,area,road_width,floors,bedroom,loc,garage
0,": 2,1 tỷ",32 m\n2,1m,2 lầu,2 phòng ngủ,"Đường Trần Bình Trọng, Phường Hải Châu I, Quận...",0
1,": 2,85 tỷ",37 m\n2,"2,5m",3 lầu,3 phòng ngủ,"Đường Nguyễn Chí Thanh, Phường Hải Châu I, Quậ...",0
2,": 2,67 tỷ",37 m\n2,3m,2 lầu,2 phòng ngủ,"Đường Hùng Vương, Phường Hải Châu I, Quận Hải ...",0
3,: 3 tỷ,59 m\n2,"2,5m",3 lầu,3 phòng ngủ,"Đường Trưng Nữ Vương, Phường Bình Hiên, Quận H...",0
4,": 2,8 tỷ",39 m\n2,2m,3 lầu,3 phòng ngủ,"Đường Hoàng Diệu, Phường Hải Châu II, Quận Hải...",0
...,...,...,...,...,...,...,...
20666,": 4,25 tỷ",97 m\n2,5m,2 lầu,3 phòng ngủ,"Đường Tô Hiệu, Phường Hòa Minh, Quận Liên Chiể...",0
20667,": 6,95 tỷ",92 m\n2,7m,3 lầu,3 phòng ngủ,"Đường Hòa Phú 28, Phường Hòa Minh, Quận Liên C...",0
20668,": 4,4 tỷ",100 m\n2,5m,3 lầu,3 phòng ngủ,"Đường Lỗ Giáng 24, Phường Hòa Xuân, Quận Cẩm L...",chỗ để xe
20670,": 3,3 tỷ",50 m\n2,3m,2 lầu,3 phòng ngủ,"Đường Lê Đình Lý, Phường Hòa Thuận Tây, Quận H...",chỗ để xe


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9831 entries, 0 to 20672
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   price       9830 non-null   object
 1   area        9830 non-null   object
 2   road_width  9825 non-null   object
 3   floors      9616 non-null   object
 4   bedroom     9741 non-null   object
 5   loc         9830 non-null   object
 6   garage      9831 non-null   object
dtypes: object(7)
memory usage: 614.4+ KB


In [143]:
def clean_price(price, area):
    if isinstance(price, str):
        price = price.replace(':', '').replace(',', '.').strip()

        if 'ngàn' in price:
            return None
        
        if 'triệu / m2' in price and isinstance(area, str):
            try:
                area_value = clean_area(area)
                price_value = float(price.replace('triệu/m2', '').strip())
                return price_value * area_value / 1000 
            except:
                return None

        if 'tỷ' in price:
            try:
                return float(price.replace('tỷ', '').strip())
            except:
                return None
        
        if '/ m2' in price:
            try:
                price_value = price.split('/')[0].strip()
                return float(price_value)
            except:
                return None

    return None
def clean_area(area):
    if isinstance(area, str):
        area = area.replace('\n', '').replace('m', '').replace('2', '').strip() 
        try:
            return float(area)
        except:
            return None
    return None

def clean_numeric_column(value):
    if isinstance(value, str):
        try:
            return float(''.join([c for c in value if c.isdigit() or c == '.']))
        except:
            return None
    return None

def clean_garage(garage):
    if isinstance(garage, str) and "0" not in garage:
        return 1
    return 0

In [144]:
df['price'] = df.apply(lambda row: clean_price(row['price'], row['area']), axis=1)
df['area'] = df['area'].apply(clean_area)
df['garage'] = df['garage'].apply(clean_garage)
df['road_width'] = df['road_width'].apply(clean_numeric_column)
df['bedroom'] = df['bedroom'].apply(clean_numeric_column)
df['floors'] = df['floors'].apply(clean_numeric_column)


In [145]:
df['bedroom'].unique()

array([  2.,   3.,   1.,   6.,   4.,   5.,  12.,  10.,  25.,   8.,  26.,
         7.,  11.,   9.,  20.,  14.,  22.,  18.,  13.,  nan,  16.,  15.,
        19.,  21.,  17.,  24.,  51.,  32.,  30.,  80.,  34.,  23.,  33.,
        56.,  27.,  28.,  36., 150.])

In [146]:
df[df['bedroom'] == 150]

,price,area,road_width,floors,bedroom,loc,garage
13540,52.0,500.0,33.0,15.0,150.0,"Đường 2/9, Phường Hòa Cường Bắc, Quận Hải Châu...",1


In [147]:
df['bedroom'] = df['bedroom'].replace(150, np.nan)

In [148]:
df['floors'].unique()

array([ 2.,  3.,  1., nan,  4.,  5., 11.,  7.,  6.,  8.,  9., 10., 13.,
       14., 18., 12., 15., 20.])

In [149]:
df['road_width'].unique()

array([  1.,  25.,   3.,   2.,   4.,  55.,  75.,  45.,  35.,   6.,  11.,
        15.,   7.,  10.,  32.,  22.,   5.,  20.,   9.,  12.,  30.,  60.,
         8.,  37.,  50.,  33.,  40., 105.,   0.,  65.,  18.,  48.,  36.,
        38., 375.,  26., 104.,  nan, 545.,  31.,  76.,  16.,  63.,  46.,
        53.,  43.,  34.,  21.,  28.,  27., 137.,  42.,  23.,  59.,  24.,
        56.,  19.,  13.,  70.,  95.,  17.,  62.])

In [150]:
df['road_width'] = df['road_width'].replace(375, np.nan)

In [151]:
df['road_width'] = df['road_width'].replace(545, np.nan)

In [152]:
df['road_width'] = df['road_width'].replace(105, np.nan)

In [153]:
df['price'].unique()

array([  2.1  ,   2.85 ,   2.67 ,   3.   ,   2.8  ,   2.4  ,   2.15 ,
         2.45 ,   2.95 ,   2.75 ,   2.25 ,   2.7  ,   1.78 ,   3.9  ,
         3.55 ,   3.95 ,   3.8  ,   2.9  ,   1.5  ,   3.5  ,   2.6  ,
         3.05 ,   2.05 ,   1.85 ,   3.6  ,   2.35 ,   2.3  ,   6.65 ,
         3.85 ,   4.6  ,   3.4  ,   4.75 ,   4.5  ,   1.99 ,   2.835,
         2.65 ,   2.5  ,   2.78 ,   2.29 ,   5.9  ,   5.2  ,   5.5  ,
         3.2  ,   7.6  ,   9.7  ,   7.   ,  23.9  ,  21.   ,  25.   ,
        27.   ,     nan, 101.   ,  60.   ,  59.   ,  40.   ,  20.   ,
        45.   ,  52.   ,  23.5  ,  50.   ,  80.   ,  18.5  ,  66.   ,
         7.9  ,   4.48 ,   4.   ,  37.   ,   4.65 ,   5.8  ,   4.25 ,
         7.5  ,   4.1  ,   6.9  ,   1.95 ,   1.75 ,   5.89 ,   5.3  ,
         5.25 ,   3.15 ,   4.3  ,   6.2  ,   4.7  ,   5.45 ,   6.3  ,
         5.   ,   3.35 ,   3.7  ,   4.35 ,   4.4  ,   4.2  ,   4.9  ,
         3.25 ,   6.6  ,   6.   ,   3.02 ,   7.95 ,   7.2  ,   7.199,
        12.499,   3.

In [154]:
# df['price']=df['price'].fillna(df['price'].mean())
# df['area']=df['area'].fillna(df['area'].mean())
df['road_width']=df['road_width'].fillna(df['road_width'].mean())
df['floors']=df['floors'].fillna(int(df['floors'].mean()))
df['bedroom']=df['bedroom'].fillna(int(df['bedroom'].mean()))

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9831 entries, 0 to 20672
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       9784 non-null   float64
 1   area        9827 non-null   float64
 2   road_width  9831 non-null   float64
 3   floors      9831 non-null   float64
 4   bedroom     9831 non-null   float64
 5   loc         9830 non-null   object 
 6   garage      9831 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 614.4+ KB


In [156]:
df=df.dropna()

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9781 entries, 0 to 20672
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       9781 non-null   float64
 1   area        9781 non-null   float64
 2   road_width  9781 non-null   float64
 3   floors      9781 non-null   float64
 4   bedroom     9781 non-null   float64
 5   loc         9781 non-null   object 
 6   garage      9781 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 611.3+ KB


In [158]:
# df.to_csv('cleaned_df_alonhadat.csv',encoding='utf-8',index=False)

In [112]:
# def remove_outlier_usingIQR(df,column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [113]:
# columns_to_clean = ['area','price']
# for column in columns_to_clean:
#     cleaned_df = remove_outlier_usingIQR(cleaned_df, column)
# cleaned_df.describe()

## Processing data relating to location

In [159]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="lengocduy169@gmail.com", timeout = 10)
geoadd = RateLimiter(geolocator.geocode, min_delay_seconds= 1)


In [162]:
df['location'] = df.loc[:, 'loc'].apply(geoadd) 

df[:20]

/tmp/ipykernel_574414/1140360068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location'] = df.loc[:, 'loc'].apply(geoadd)


,price,area,road_width,floors,bedroom,loc,garage,location
0,2.10,3.0,1.0,2.0,2.0,"Đường Trần Bình Trọng, Phường Hải Châu I, Quận...",0,"(Đường Trần Bình Trọng, Phường Hải Châu I, Quậ..."
1,2.85,37.0,25.0,3.0,3.0,"Đường Nguyễn Chí Thanh, Phường Hải Châu I, Quậ...",0,"(Đường Nguyễn Chí Thanh, Phường Hải Châu I, Qu..."
2,2.67,37.0,3.0,2.0,2.0,"Đường Hùng Vương, Phường Hải Châu I, Quận Hải ...",0,"(Đường Hùng Vương, Phường Hải Châu I, Quận Hải..."
3,3.00,59.0,25.0,3.0,3.0,"Đường Trưng Nữ Vương, Phường Bình Hiên, Quận H...",0,"(Đường Trưng Nữ Vương, Phường Bình Hiên, Quận ..."
4,2.80,39.0,2.0,3.0,3.0,"Đường Hoàng Diệu, Phường Hải Châu II, Quận Hải...",0,None
5,2.40,40.0,25.0,2.0,2.0,"Đường Hải Phòng, Phường Hải Châu I, Quận Hải C...",0,"(Đường Hải Phòng, Phường Hải Châu I, Quận Hải ..."
6,2.15,40.0,25.0,2.0,3.0,"Đường Ông Ích Đường, Phường Hòa Thọ Đông, Quận...",0,"(Đường Ông Ích Đường, Phường Hoà Thọ Đông, Quậ..."
7,2.45,34.0,25.0,3.0,2.0,"Đường Cô Bắc, Phường Hải Châu II, Quận Hải Châ...",0,"(Đường Cô Bắc, Phường Hải Châu II, Quận Hải Ch..."
8,2.80,45.0,25.0,2.0,2.0,"Đường Cô Bắc, Phường Hải Châu II, Quận Hải Châ...",0,"(Đường Cô Bắc, Phường Hải Châu II, Quận Hải Ch..."
9,2.95,43.0,3.0,3.0,3.0,"Đường Ông Ích Khiêm, Phường Nam Dương, Quận Hả...",0,"(Đường Ông Ích Khiêm, Phường Hải Châu II, Quận..."


In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9781 entries, 0 to 20672
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       9781 non-null   float64
 1   area        9781 non-null   float64
 2   road_width  9781 non-null   float64
 3   floors      9781 non-null   float64
 4   bedroom     9781 non-null   float64
 5   loc         9781 non-null   object 
 6   garage      9781 non-null   int64  
 7   location    8918 non-null   object 
dtypes: float64(5), int64(1), object(2)
memory usage: 945.8+ KB


In [164]:
df.loc[:,'latitude'] = df.loc[:,'location'].apply(lambda loc: loc.latitude if loc else None)
df.loc[:,'longitude'] = df.loc[:,'location'].apply(lambda loc: loc.longitude if loc else None)

/tmp/ipykernel_574414/438653058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'latitude'] = df.loc[:,'location'].apply(lambda loc: loc.latitude if loc else None)
/tmp/ipykernel_574414/438653058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'longitude'] = df.loc[:,'location'].apply(lambda loc: loc.longitude if loc else None)


In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9781 entries, 0 to 20672
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       9781 non-null   float64
 1   area        9781 non-null   float64
 2   road_width  9781 non-null   float64
 3   floors      9781 non-null   float64
 4   bedroom     9781 non-null   float64
 5   loc         9781 non-null   object 
 6   garage      9781 non-null   int64  
 7   location    8918 non-null   object 
 8   latitude    8918 non-null   float64
 9   longitude   8918 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 1.1+ MB


In [170]:
df[['Street', 'Ward', 'District']] = df['loc'].str.split(',', n=2)

,price,area,road_width,floors,bedroom,garage,latitude,longitude,Street,Ward,District
0,2.10,3.0,1.0,2.0,2.0,0,16.066137,108.219421,Đường Trần Bình Trọng,Phường Hải Châu I,"Quận Hải Châu, Đà Nẵng"
1,2.85,37.0,25.0,3.0,3.0,0,16.073207,108.220864,Đường Nguyễn Chí Thanh,Phường Hải Châu I,"Quận Hải Châu, Đà Nẵng"
2,2.67,37.0,3.0,2.0,2.0,0,16.068601,108.222606,Đường Hùng Vương,Phường Hải Châu I,"Quận Hải Châu, Đà Nẵng"
3,3.00,59.0,25.0,3.0,3.0,0,16.059001,108.221297,Đường Trưng Nữ Vương,Phường Bình Hiên,"Quận Hải Châu, Đà Nẵng"
4,2.80,39.0,2.0,3.0,3.0,0,NaN,NaN,Đường Hoàng Diệu,Phường Hải Châu II,"Quận Hải Châu, Đà Nẵng"
...,...,...,...,...,...,...,...,...,...,...,...
20666,4.25,97.0,5.0,2.0,3.0,0,16.063051,108.165838,Đường Tô Hiệu,Phường Hòa Minh,"Quận Liên Chiểu, Đà Nẵng"
20667,6.95,9.0,7.0,3.0,3.0,0,16.071464,108.160096,Đường Hòa Phú 28,Phường Hòa Minh,"Quận Liên Chiểu, Đà Nẵng"
20668,4.40,100.0,5.0,3.0,3.0,1,15.989052,108.209341,Đường Lỗ Giáng 24,Phường Hòa Xuân,"Quận Cẩm Lệ, Đà Nẵng"
20670,3.30,50.0,3.0,2.0,3.0,1,16.051228,108.209638,Đường Lê Đình Lý,Phường Hòa Thuận Tây,"Quận Hải Châu, Đà Nẵng"


In [171]:
df[['District', 'City']] = df['District'].str.rsplit(',', n=1, expand=True)

In [188]:
from geopy.distance import geodesic
beach = (16.070249992972627, 108.2461155487451) #Mi Khe
center = (16.061928844525685, 108.22244559548565) #Cau Rong
airport = (16.05397046603145, 108.20445571667258) #San bay
def calculate_distance(row,loc):
    return geodesic((row['latitude'],row['longitude']), loc).kilometers


In [191]:
df['distance_to_beach'] = df.apply(lambda row: calculate_distance(row, beach), axis=1)
df['distance_to_center'] = df.apply(lambda row: calculate_distance(row, center), axis=1)
df['distance_to_airport'] = df.apply(lambda row: calculate_distance(row, airport), axis=1)

/tmp/ipykernel_574414/633411228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance_to_beach'] = df.apply(lambda row: calculate_distance(row, beach), axis=1)
/tmp/ipykernel_574414/633411228.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance_to_center'] = df.apply(lambda row: calculate_distance(row, center), axis=1)
/tmp/ipykernel_574414/633411228.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [198]:
df = df.drop(columns=['Street','Ward','City'])

In [200]:
df['District'].unique()

array([' Quận Hải Châu', ' Quận Cẩm Lệ', ' Quận Thanh Khê',
       ' Huyện Hòa Vang', ' Quận Sơn Trà', ' Quận Liên Chiểu',
       ' Quận Ngũ Hành Sơn'], dtype=object)

In [201]:
mapping = {
    ' Quận Hải Châu': 1,
    ' Quận Cẩm Lệ': 2,
    ' Quận Thanh Khê': 3,
    ' Huyện Hòa Vang': 4,
    ' Quận Sơn Trà': 5,
    ' Quận Liên Chiểu': 6,
    ' Quận Ngũ Hành Sơn': 7
}

In [202]:
df['District'] = df['District'].map(mapping)

In [204]:
df['District'].unique()

array([1, 2, 3, 4, 5, 6, 7])

In [206]:
df

,price,area,road_width,floors,bedroom,garage,latitude,longitude,District,distance_to_beach,distance_to_center,distance_to_airport
0,2.10,3.0,1.0,2.0,2.0,0,16.066137,108.219421,1,2.892327,0.567063,2.092050
1,2.85,37.0,25.0,3.0,3.0,0,16.073207,108.220864,1,2.721554,1.259400,2.759275
2,2.67,37.0,3.0,2.0,2.0,0,16.068601,108.222606,1,2.522046,0.738570,2.528471
3,3.00,59.0,25.0,3.0,3.0,0,16.059001,108.221297,1,2.932883,0.346556,1.886090
5,2.40,40.0,25.0,2.0,2.0,0,16.072715,108.220794,1,2.723046,1.206563,2.712644
...,...,...,...,...,...,...,...,...,...,...,...,...
20666,4.25,97.0,5.0,2.0,3.0,0,16.063051,108.165838,6,8.626475,6.058356,4.252655
20667,6.95,9.0,7.0,3.0,3.0,0,16.071464,108.160096,6,9.204821,6.754304,5.126150
20668,4.40,100.0,5.0,3.0,3.0,1,15.989052,108.209341,2,9.809419,8.185563,7.202866
20670,3.30,50.0,3.0,2.0,3.0,1,16.051228,108.209638,1,4.434577,1.811174,0.632172


In [207]:
df.to_csv("prepared_loc.csv",encoding='utf-8',index=False)

## IMPORTING DATA INTO DATABASE

In [208]:
import pymysql


timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="Housing_Real_Estate",
  host="mysql-for-ady201m-project-ady201m-nhom5.e.aivencloud.com",
  password="AVNS_Z2RBipwQfoNZD7N0O11",
  read_timeout=timeout,
  port=22064,
  user="avnadmin",
  write_timeout=timeout,
)


In [211]:
cursor = connection.cursor()


In [210]:
df.columns

Index(['price', 'area', 'road_width', 'floors', 'bedroom', 'garage',
       'latitude', 'longitude', 'District', 'distance_to_beach',
       'distance_to_center', 'distance_to_airport'],
      dtype='object')

In [213]:
cursor.execute("CREATE DATABASE IF NOT EXISTS Housing_Real_Estate")
    
cursor.execute("USE Housing_Real_Estate")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Housing_Data (
        id INT PRIMARY KEY AUTO_INCREMENT,
        price FLOAT,
        area FLOAT,
        road_width FLOAT,
        floors FLOAT,
        bedroom INT,
        garage INT,
        latitude FLOAT,
        longitude FLOAT,
        district INT,
        distance_to_beach FLOAT,
        distance_to_center FLOAT,
        distance_to_airport FLOAT
    )
""")

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Housing_Data
        (price, area, road_width, floors, bedroom, garage, latitude,longitude,district,distance_to_beach,distance_to_center,distance_to_airport)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['price'], 
        row['area'], 
        row['road_width'], 
        row['floors'], 
        row['bedroom'], 
        row['garage'], 
        row['latitude'],
        row['longitude'],
        row['District'],
        row['distance_to_beach'],
        row['distance_to_center'],
        row['distance_to_airport'],
    ))

In [214]:
connection.commit()

In [217]:
cursor.execute("USE Housing_Real_Estate")
cursor.execute("SELECT * FROM Housing_Data")
results = cursor.fetchall()
for result in results:
    print(result)

{'id': 1, 'price': 2.1, 'area': 3.0, 'road_width': 1.0, 'floors': 2.0, 'bedroom': 2, 'garage': 0, 'latitude': 16.0661, 'longitude': 108.219, 'district': 1, 'distance_to_beach': 2.89233, 'distance_to_center': 0.567063, 'distance_to_airport': 2.09205}
{'id': 2, 'price': 2.85, 'area': 37.0, 'road_width': 25.0, 'floors': 3.0, 'bedroom': 3, 'garage': 0, 'latitude': 16.0732, 'longitude': 108.221, 'district': 1, 'distance_to_beach': 2.72155, 'distance_to_center': 1.2594, 'distance_to_airport': 2.75928}
{'id': 3, 'price': 2.67, 'area': 37.0, 'road_width': 3.0, 'floors': 2.0, 'bedroom': 2, 'garage': 0, 'latitude': 16.0686, 'longitude': 108.223, 'district': 1, 'distance_to_beach': 2.52205, 'distance_to_center': 0.73857, 'distance_to_airport': 2.52847}
{'id': 4, 'price': 3.0, 'area': 59.0, 'road_width': 25.0, 'floors': 3.0, 'bedroom': 3, 'garage': 0, 'latitude': 16.059, 'longitude': 108.221, 'district': 1, 'distance_to_beach': 2.93288, 'distance_to_center': 0.346556, 'distance_to_airport': 1.8860

In [ ]:
connection.close()